In [5]:
from google.colab import drive
from google.colab import files
import sys
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [6]:
import cv2
import numpy as np
import os
import skimage
from skimage.metrics import structural_similarity as ssim
import cv2
import numpy as np

In [7]:
def get_accuracy(input, ground_truth):
    input = cv2.imread(input)
    ground_truth = cv2.imread(ground_truth)
    input_points = np.size(input)
    ground_truth_points = np.size(ground_truth)
    before_gray = cv2.cvtColor(input, cv2.COLOR_BGR2GRAY)
    after_gray = cv2.cvtColor(ground_truth, cv2.COLOR_BGR2GRAY)
    (score, diff) = ssim(before_gray, after_gray, full=True)
    print("Image similarity", score)

    # The diff image contains the actual image differences between the two images
    # and is represented as a floating point data type in the range [0,1] 
    # so we must convert the array to 8-bit unsigned integers in the range
    # [0,255] before we can use it with OpenCV
    diff = (diff * 255).astype("uint8")

    # Threshold the difference image, followed by finding contours to
    # obtain the regions of the two input images that differ
    thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    contours = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0] if len(contours) == 2 else contours[1]

    mask = np.zeros(input.shape, dtype='uint8')
    filled_after = ground_truth.copy()

    for c in contours:
        area = cv2.contourArea(c)
        if area > 40:
            x,y,w,h = cv2.boundingRect(c)
            cv2.rectangle(input, (x, y), (x + w, y + h), (36,255,12), 2)
            cv2.rectangle(ground_truth, (x, y), (x + w, y + h), (36,255,12), 2)
            cv2.drawContours(mask, [c], 0, (0,255,0), -1)
            cv2.drawContours(filled_after, [c], 0, (0,255,0), -1)
    common_points = input_points- np.count_nonzero(mask)
    print(common_points)
    print(np.count_nonzero(ground_truth))
    recall = 1.0 * common_points / np.size(input) if common_points>0 else 0
    precision = 1.0 * common_points / np.size(input) if common_points>0 else 0
    f1_score = 2*precision*recall/(precision + recall)
    print('Precision: ' + str(precision))
    print('Recall: ' + str(recall))
    print('F1-Socre: ' + str(f1_score))
    while (1):
        cv2.imshow('input', input)
        cv2.imshow('ground_truth', ground_truth)
        cv2.imshow('diff',diff)
        cv2.imshow('mask',mask)
        cv2.imshow('filled after',filled_after)
        cv2.waitKey(0)
        k = cv2.waitKey(1)
        # break
        if k == 27:
            break
    cv2.destroyAllWindows()

def get_label(img, label):
    clutter = [0,0,0]
    building = [128,0,0]
    grass = [0,128,0]
    pond = [0,0,128]
    road = [128,64,128]
    dirt = [60,40,222]
    tree = [128,128,0]
    vehicle = [64,0,128]
    sign = [192,128,128]
    Road = [0,0,255]
    img = cv2.imread(img)
    if (label == 'g'):
        mask = np.all(img == grass,  axis=-1)
        img[~mask] = [0,0,0]
    elif (label == 'c'):
        mask = np.all(img == clutter,  axis=-1)
        img[~mask] = [0,0,0]
    elif (label == 'p'):
        mask = np.all(img == pond,  axis=-1)
        img[~mask] = [0,0,0]
    elif (label == 'r'):
        mask = np.all(img == road,  axis=-1)
        img[~mask] = [0,0,0]
    return img
def accuracy_per_label(input, ground_truth, label):
    recall = 0
    precision = 0
    f1_score = 0
    if (label.lower() == 'g'):
        input = get_label(input, label.lower())
        ground_truth = get_label(ground_truth, label.lower())
        input[input > 0] = 1
        ground_truth[ground_truth != 0] = 1
        ones = (input==1)[ground_truth==1]
        common_points = sum(ones)
        recall = 1.0 * common_points / np.count_nonzero(ground_truth) if common_points>0 else 0
        precision = 1.0 * common_points / np.count_nonzero(input) if common_points>0 else 0
        f1_score = 2*precision*recall/(precision + recall)
        print("--------------")
        print("Grass")
        print('Precision: ' + str(precision))
        print('Recall: ' + str(recall))
        print('F1-Score: ' + str(f1_score))
    elif (label.lower() == 'c'):
        input = get_label(input, label.lower())
        ground_truth = get_label(ground_truth, label.lower())
        input[input > 0] = 1
        ground_truth[ground_truth != 0] = 1
        ones = (input==1)[ground_truth==1]
        common_points = sum(ones)
        recall = 1.0 * common_points / np.count_nonzero(ground_truth) if common_points>0 else 0
        precision = 1.0 * common_points / np.count_nonzero(input) if common_points>0 else 0
        f1_score = 2*precision*recall/(precision + recall)
        print("--------------")
        print("Clutter")
        print('Precision: ' + str(precision))
        print('Recall: ' + str(recall))
        print('F1-Score: ' + str(f1_score))
    elif (label.lower() == 'p'):
        input = get_label(input, label.lower())
        ground_truth = get_label(ground_truth, label.lower())
        input[input > 0] = 1
        ground_truth[ground_truth != 0] = 1
        ones = (input==1)[ground_truth==1]
        common_points = sum(ones)
        recall = 1.0 * common_points / np.count_nonzero(ground_truth) if common_points>0 else 0
        precision = 1.0 * common_points / np.count_nonzero(input) if common_points>0 else 0
        f1_score = 2*precision*recall/(precision + recall)
        print("--------------")
        print("Pond")
        print('Precision: ' + str(precision))
        print('Recall: ' + str(recall))
        print('F1-Score: ' + str(f1_score))
    elif (label.lower() == 'r'):
        input = get_label(input, label.lower())
        ground_truth = get_label(ground_truth, label.lower())
        input[input > 0] = 1
        ground_truth[ground_truth != 0] = 1
        ones = (input==1)[ground_truth==1]
        common_points = sum(ones)
        recall = 1.0 * common_points / np.count_nonzero(ground_truth) if common_points>0 else 0
        precision = 1.0 * common_points / np.count_nonzero(input) if common_points>0 else 0
        f1_score = 2*precision*recall/(precision + recall)
        print("--------------")
        print("Road")
        print('Precision: ' + str(precision))
        print('Recall: ' + str(recall))
        print('F1-Score: ' + str(f1_score))
    return recall, precision, f1_score


In [8]:
directory = '/content/drive/MyDrive/Unet/GDOT-images/NCAT_UNet_preprocessed_images/Test_3_label'
count = 0
total_recall = 0
total_precision = 0
total_f1_score = 0
total = 0
for filename in os.listdir(directory):
    total = total + 1
    print(os.path.join(directory, filename))
    print('/content/drive/MyDrive/Unet/results/' + str(count) + '_predict.png')
    recall, precision, f1_score = accuracy_per_label('/content/drive/MyDrive/Unet/results/' + str(count) + '_predict.png', os.path.join(directory, filename), 'g')
    count = count + 1
    total_recall = total_recall + recall
    total_precision = total_precision + precision
    total_f1_score = total_f1_score + f1_score
    print("Average precision: " + str(total_precision/total))
    print("Average recall: " + str(total_recall/total))
    print("Average f1-score: " + str(total_f1_score/total))
print("Average precision: " + str(total_precision/total))
print("Average recall: " + str(total_recall/total))
print("Average f1-score: " + str(total_f1_score/total))

/content/drive/MyDrive/Unet/GDOT-images/NCAT_UNet_preprocessed_images/Test_3_label/DJI_0203.png
/content/drive/MyDrive/Unet/results/0_predict.png
--------------
Grass
Precision: 0.9723101876882676
Recall: 0.8538628501661806
F1-Score: 0.9092452148790177
Average precision: 0.9723101876882676
Average recall: 0.8538628501661806
Average f1-score: 0.9092452148790177
/content/drive/MyDrive/Unet/GDOT-images/NCAT_UNet_preprocessed_images/Test_3_label/DJI_0204.png
/content/drive/MyDrive/Unet/results/1_predict.png
--------------
Grass
Precision: 0.9593796773657076
Recall: 0.8589242508840499
F1-Score: 0.9063770449947954
Average precision: 0.9658449325269876
Average recall: 0.8563935505251152
Average f1-score: 0.9078111299369065
/content/drive/MyDrive/Unet/GDOT-images/NCAT_UNet_preprocessed_images/Test_3_label/DJI_0205.png
/content/drive/MyDrive/Unet/results/2_predict.png
--------------
Grass
Precision: 0.9548623687069903
Recall: 0.8565568777159567
F1-Score: 0.9030421099955044
Average precision: 0.

TypeError: ignored